In [44]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
NLTK: 3.4.4
Scikit-learn: 0.21.3
Pandas: 0.25.0
Numpy: 1.17.0


In [45]:
# Carregando dados
import pandas as pd

df = pd.read_csv('../data/dataset_classes_dez_mais.txt', header=None, encoding='utf-8', sep='\t')
df.fillna('x', inplace=True)
#df_classes_raw = pd.read_csv('data/classes_raw.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null object
1    793 non-null object
dtypes: object(2)
memory usage: 12.5+ KB
None
       0                                                  1
0    stj  pretensa estadual deduz apel extrem esbarr ved...
1      n  embarg insist mesm razo recurs apresent oposic...
2      n  acorda ora recorr nao neg vigenc direit federa...
3  civil  tribunal justic assent compreensa part det leg...
4      n  segund jurisprudenc superior tribunal justic a...


In [71]:
# verificação das classes dos acórdãos

classes = df[0]
print(classes)
print(len(classes.value_counts()))
classes.value_counts().to_frame().style.bar()

0         stj
1           n
2           n
3       civil
4           n
        ...  
788    embarg
789     civil
790         n
791    outros
792         n
Name: 0, Length: 793, dtype: object
11


,0
embarg,195
civil,194
n,133
outros,100
recurs,65
process,45
regimental,24
stj,23
agrav,7
sumul,5


In [47]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes
encoder = LabelEncoder()
classes = encoder.fit_transform(classes)

print(classes[:10])
len(classes)

[9 4 4 1 4 5 6 6 4 0]


793

In [48]:
# criação da lista de ementas

ementas = df[1]
print(ementas[:10])

0    pretensa estadual deduz apel extrem esbarr ved...
1    embarg insist mesm razo recurs apresent oposic...
2    acorda ora recorr nao neg vigenc direit federa...
3    tribunal justic assent compreensa part det leg...
4    segund jurisprudenc superior tribunal justic a...
5    sobrest recurs especial ate julgament recurs e...
6    inadmissivel exigenc recolh reu prisa requisit...
7    dentr limit leg vez caracteriz reincidenc agra...
8    sobr prescrica aca repetica indebit tributari ...
9    tratas agrav instrument interpost contr decisa...
Name: 1, dtype: object


In [49]:
# uso de word2vec para a extração de features

from gensim.models import Word2Vec

ementas_list = [word for word in ementas.iteritems()]
ementas_list_list = []
for doc in ementas_list:
    ementas_list_list.append(doc[1].split())

X = Word2Vec(ementas_list_list, min_count=2)
X.wv.init_sims() # para inicializar model.vw.syn0norm, necessário para o cálculo da média da
print(X)

Word2Vec(vocab=3183, size=100, alpha=0.025)


In [50]:
# funções para calcular a média vetorial das palavras de cada ementa
import numpy as np

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        print("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [51]:
# função para tokenizar as palavras com frequência maior que 2
import nltk


def w2v_tokenize_text(text):    
    tokens = []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [65]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


train, test = train_test_split(df, test_size=0.2, random_state=0)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values

X_train = word_averaging_list(X.wv,train_tokenized)
X_test = word_averaging_list(X.wv,test_tokenized)

y_train = train[0]
y_test = test[0]
print("train: {},{}".format(len(X_train),len(y_train)))

cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
cannot compute similarity with no input %s []
train: 634,634


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

In [61]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 32.075471698113205


In [55]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

              precision    recall  f1-score   support

       agrav       0.00      0.00      0.00         1
       civil       0.43      0.45      0.44        40
      embarg       0.25      0.80      0.39        35
           n       0.25      0.03      0.05        35
      outros       1.00      0.16      0.27        19
     process       0.00      0.00      0.00         7
      recurs       0.00      0.00      0.00        13
  regimental       0.00      0.00      0.00         4
         stj       0.00      0.00      0.00         3
       sumul       0.00      0.00      0.00         2

    accuracy                           0.31       159
   macro avg       0.19      0.14      0.11       159
weighted avg       0.34      0.31      0.24       159

31.446540880503143


/home/audora/tcc/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
# usando undersampling
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy='not minority', random_state=0)
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
print("train: {},{}".format(len(X_resampled),len(y_resampled)))

series = []
for y in y_resampled:
    series.append(y)
    
nltk.FreqDist(series).most_common(20)

train: 22,22


[('agrav', 2),
 ('civil', 2),
 ('embarg', 2),
 ('especial', 2),
 ('n', 2),
 ('outros', 2),
 ('process', 2),
 ('recurs', 2),
 ('regimental', 2),
 ('stj', 2),
 ('sumul', 2)]

In [75]:
# Fazendo a mesma análisa usando undersampling

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_resampled, y_resampled)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 3.7735849056603774
Decision Tree Accuracy: 8.176100628930817
Random Forest Accuracy: 10.062893081761008
Logistic Regression Accuracy: 10.062893081761008
SGD Classifier Accuracy: 1.8867924528301887
Naive Bayes Accuracy: 5.660377358490567


In [76]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced'),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_resampled, y_resampled)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 5.660377358490567


In [59]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

              precision    recall  f1-score   support

       agrav       1.00      1.00      1.00         1
       civil       0.28      0.97      0.44        40
      embarg       0.00      0.00      0.00        35
           n       0.40      0.06      0.10        35
      outros       0.21      0.16      0.18        19
     process       0.00      0.00      0.00         7
      recurs       0.00      0.00      0.00        13
  regimental       0.00      0.00      0.00         4
         stj       0.00      0.00      0.00         3
       sumul       0.00      0.00      0.00         2

    accuracy                           0.28       159
   macro avg       0.19      0.22      0.17       159
weighted avg       0.19      0.28      0.16       159

28.30188679245283


/home/audora/tcc/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
